# 1. Instalar y cargar librería

In [1]:
# instalar previamente
!pip install plotly
!pip install cufflinks

In [2]:
import pandas as pd
import cufflinks as cf
from IPython.display import display,HTML
import os
import numpy as np
import time
inicio_todo_el_proceso = time.time()
cf.set_config_file(sharing = 'public',theme = 'ggplot', offline = True)

# 2. Cargar base de datos y definir

In [3]:
# Acá debes cargar la o las bases de datos y definir la abreviación

abrev = 'RELIME'

fuente_artículos = 'Scopus'

#relime
url1='https://www.dropbox.com/s/uzlmcqrs4o9o0s3/scopus%20-%202022-12-28theatreOR%20%20perfomance_ethnic.csv?dl=1'
url1='https://www.dropbox.com/s/k6v8jsn9wyy0j80/1.3_latam_modelaci%C3%B3n_rev_mat%28110%29.csv?dl=1'
url1 ='https://www.dropbox.com/s/h7lu7uzs99gxnan/scopus%20-%202022-12-29T114812.218_UPLA.csv?dl=1'
url1='https://www.dropbox.com/s/we8yeu8ub3r4k0u/scopus%20-%202022-12-03T175540.018_RELIME.csv?dl=1'
url1='https://www.dropbox.com/s/co1z8dgfhpwxsbx/scopus_2000_rev_Education_Policy_Analysis_Archives.csv?dl=1'
url1='https://www.dropbox.com/s/bfi5hkuhlioa8xp/scopus_multipl_disabilit_policy_public_educat.csv?dl=1'

url1='https://www.dropbox.com/s/we8yeu8ub3r4k0u/scopus%20-%202022-12-03T175540.018_RELIME.csv?dl=1'

url=url1



dfA=pd.read_csv(url,on_bad_lines='skip')

##### para unir varios df en uno#####

#archivos=[url1,url2]
#dataframes = []
#for archivo in archivos:
#    df = pd.read_csv(archivo)
#    dataframes.append(df)

#df_final = pd.concat(dataframes)
#dfA=df_final
#dfA=pd.read_csv(url,on_bad_lines='skip')

print(len(dfA))



220


In [4]:
dfA.head(1)

,Authors,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,Page end,...,Link,Abstract,Author Keywords,Index Keywords,Language of Original Document,Document Type,Publication Stage,Open Access,Source,EID
0,"Kuzniak A., Richard P.R.",24465721100;57197042114;,Spaces for mathematical work: Viewpoints and p...,2014,Revista Latinoamericana de Investigacion en Ma...,17,4,NaN,5.0,39.0,...,https://www.scopus.com/inward/record.uri?eid=2...,This special issue of Revisla Latinoamericana ...,NaN,NaN,English; French; Spanish,Article,Final,"All Open Access, Gold",Scopus,2-s2.0-85012050295


In [5]:
dfA.columns.values


array(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title',
       'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Abstract',
       'Author Keywords', 'Index Keywords',
       'Language of Original Document', 'Document Type',
       'Publication Stage', 'Open Access', 'Source', 'EID'], dtype=object)

# 3 Campos a analizar y método a utilizar

In [6]:
dic_campo_a_analizar = {1:'key_words',2:'title',3:'abstract',4:'title_key',5:'title_abs',6:'key_abs',7:'title_abs_key'}

#Cambiar número para ver campo a analizar

dic_método_a_utilizar = {1:'lemmatize',2:'stemming'}

método=1
campo_a_analizar = 1
template_elegido = 'seaborn'
print(dic_campo_a_analizar)
print(dic_campo_a_analizar.get(campo_a_analizar))


{1: 'key_words', 2: 'title', 3: 'abstract', 4: 'title_key', 5: 'title_abs', 6: 'key_abs', 7: 'title_abs_key'}
key_words


In [7]:
!pip install wget

# 4 Creación de carpeta donde se guardará la información

In [8]:
from datetime import datetime

now = datetime.now()


if not os.path.exists(abrev):
        os.mkdir(abrev)
        
print(now)

2023-01-12 12:55:22.880417


In [9]:

ruta_archivo = abrev+'/'
from urllib import request

# Definimos la URL del archivo a descargar

remote_url1 = "https://www.dropbox.com/s/alwmw7pt709tats/apalike-es.bst?dl=1"
remote_url2 = "https://www.dropbox.com/s/7dey16kws238ex0/referencias.bib?dl=1"

# Definimos el nombre del archivo local a guardar

local_file1 = 'apalike-es.bst' 
local_file2 = "referencias.bib"

# Se realiza la descarga y se guarda el archivo de manera local

request.urlretrieve(remote_url1, ruta_archivo+local_file1)
request.urlretrieve(remote_url2, ruta_archivo+local_file2)

('RELIME-RIITMA/referencias.bib', <http.client.HTTPMessage at 0x7fc87ac9e4f0>)

## 4.1 Creación de funciones para escribir entorno latex

In [10]:


def entorno_abrir_tex(ruta_archivo, nombre_archivo):
    file = open(ruta_archivo + nombre_archivo + ".tex", "w")
    file.write("%comienza el texto" + os.linesep)
    file.write("\n" + os.linesep)

def entorno_abrir_bib(ruta_archivo, nombre_archivo):
    file = open(ruta_archivo + nombre_archivo + ".bib", "w")
    file.write("%comienza el texto" + os.linesep)
    file.write("\n" + os.linesep)


def entorno_abrir_documento_tex(ruta_archivo, nombre_archivo):
    file = open(ruta_archivo + nombre_archivo + ".tex", "a")
    file.write('\\'+ "begin{document}" + os.linesep)
    file.write("\n" + os.linesep)

def entorno_cerrar_documento_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\'+ 'end{document}'+ os.linesep)


def entorno_figura_tex(ruta_archivo, nombre_archivo,nombre_archivo_figura, leyenda):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\'+ "begin{figure}[h!]" + os.linesep)
        file.write('\\'+"centering" + os.linesep)
        file.write("\includegraphics[width=0.7" + '\\' + "textwidth]{"+ nombre_archivo_figura +".pdf}" + os.linesep)
        file.write('\\'+ "caption{" + leyenda + '}'  + os.linesep)
        file.write('\\' + 'label{fig:' + nombre_archivo_figura + '}' + os.linesep)
        file.write('\\'+ "end{figure}" + os.linesep)
        file.write("\n" + os.linesep) 



def entorno_section_tex(ruta_archivo, nombre_archivo, nombre_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write('\\'+ 'section{' + nombre_sección +  '}' + os.linesep)
        file.write("\n" + os.linesep)
    

def entorno_sub_section_tex(ruta_archivo, nombre_archivo, nombre_sub_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write('\\'+ 'subsection{' + nombre_sub_sección +  '}' + os.linesep)
        file.write("\n" + os.linesep)
        
def entorno_sub_sub_section_tex(ruta_archivo, nombre_archivo, nombre_sub_sub_sección):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write('\\'+ 'subsubsection{' + nombre_sub_sub_sección +  '}' + os.linesep)
        file.write("\n" + os.linesep)

    
def entorno_itemize_tex(ruta_archivo, nombre_archivo,lista_para_itemize):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write('\\'+ 'begin{itemize}'+ os.linesep)
        for x in lista_para_itemize:
            file.write('\\'+ 'item ' + x + os.linesep)
        file.write('\\'+ 'end{itemize}'+ os.linesep)
        file.write("\n" + os.linesep)
        
def entorno_enumerate_tex(ruta_archivo, nombre_archivo,lista_para_enumerate):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file.write('\\'+ 'begin{enumerate}'+ os.linesep)
        for x in lista_para_enumerate:
            file.write('\\'+ 'item ' + x + os.linesep)
        file.write('\\'+ 'end{enumerate}'+ os.linesep)
        file.write("\n" + os.linesep)
      
    
def entorno_nueva_página_tex(ruta_archivo, nombre_archivo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write('\\'+ 'newpage'+ os.linesep)
        file.write("\n" + os.linesep)

# Abrir el archivo en modo escritura
            
    
def etorno_inicio_documento_tex(ruta_archivo, nombre_archivo,título):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo  + nombre_archivo + ".tex", "a")
        file.write('\\documentclass[12pt,a4paper]{article}\n')
        file.write('\\usepackage[utf8]{inputenc}\n')
        file.write('\\usepackage[spanish, es-tabla]{babel}\n')
        file.write('\\usepackage[hidelinks,colorlinks=true,citecolor=blue, backref=page]{hyperref}\n')
        file.write('\\usepackage{amsmath}\n')
        file.write('\\usepackage{amsfonts}\n')
        file.write('\\usepackage{multirow}\n')
        file.write('\\usepackage{booktabs}\n')
        file.write('\\usepackage{apacite}\n')
        file.write('\\usepackage{multirow}\n')
        file.write('\\usepackage{natbib}\n')
        file.write('\\usepackage{amssymb}\n')
        file.write('\\usepackage{graphicx}\n')
        file.write('\\usepackage[width=18.00cm, height=27.00cm]{geometry}\n')
        file.write('\\title{'+título+'}\n')
        file.write('\\begin{document}\n')
        file.write('\\maketitle \n')
        file.write('%\\end{document}\n')

        
        
def parrafo_tex(ruta_archivo, nombre_archivo, parrafo):
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        file = open(ruta_archivo + nombre_archivo + ".tex", "a")
        file.write(parrafo + os.linesep)
        file.write("\n" + os.linesep)
        

             
def agregar_tabla_latex(ruta_archivo, nombre_archivo, tabla):
    # Abrir el archivo en modo escritura
    with open(ruta_archivo + nombre_archivo + ".tex", "a") as file:
        # Escribir la tabla al archivo
        file.write(tabla)

        


In [11]:
def crear_tabla_latex(latex_table, caption, label):
    # Agregar el código LaTeX para el entorno table
    table = '\\begin{table}[h!]\n'
    table += '\\centering\n'
    table += '\\caption{' + caption + '}\n'
    table += '\\label{' + label + '}\n'
    table += latex_table
    table += '\\end{table}'
    
    # Devolver el código LaTeX del entorno table
    return table


## Referencias bibliográficas

## 4.2 Primera parte del documento latex

In [12]:
ruta_archivo = abrev+'/'
nombre_archivo_tex = '01informe_'+abrev+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
entorno_abrir_tex(ruta_archivo, nombre_archivo_tex)
título = 'Espacio Rizomático Conceptual de \\textbf{' + abrev +'}'
etorno_inicio_documento_tex(ruta_archivo, nombre_archivo_tex,título)
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Introducción')
lista_párrafo1 = ['Este documento muestra el análisis en base al trabajo de \\cite{Gaona2023}. Se realizó a partir de una base de datos con ',
            str(len(dfA)),
            ' artículos. Estos artículos pertenecen a ',
            str(len(dfA['Source title'].unique())) 
]
if len(dfA['Source title'].unique())==1:
    lista_párrafo1_1 = ' fuente documental.'
else:
    lista_párrafo1_1 = ' fuentes documentales diferentes.'

lista_párrafo1.append(lista_párrafo1_1) 
lista_párrafo1 = lista_párrafo1 + [' Los artículos fueron publicados ']
                                   
if len(dfA['Year'].unique())==1:
    lista_párrafo1_2 = ' el año '+ str(dfA['Year'].unique()) +'.'
else:
    lista_párrafo1_2 = ' entre los años ' + str(dfA['Year'].min()) + ' y ' + str(dfA['Year'].max()) +'.'
    
lista_párrafo1.append(lista_párrafo1_2) 

lista_párrafo1 = lista_párrafo1 + [' La información se extrajo de',
                                  fuente_artículos+'.']




párrafo1=' '.join(lista_párrafo1)
print(párrafo1)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo1)




Este documento muestra el análisis en base al trabajo de \cite{Gaona2023}. Se realizó a partir de una base de datos con  220  artículos. Estos artículos pertenecen a  1  fuente documental.  Los artículos fueron publicados   entre los años 2009 y 2022.  La información se extrajo de Scopus.


In [13]:
## Referencias

In [14]:
dfB = dfA.copy()

dfB.rename(columns={'Year': 'Año'}, inplace=True)

dfA_summary = dfB.groupby('Año')['Cited by'].agg(['size', 'sum'])
dfA_summary.rename(columns={'size': 'Nº artículos', 'sum': 'Nº de  citas'}, inplace=True)
dfA_summary.reset_index(inplace=False)
dfA_summary.sort_values(by='Año', ascending=True, inplace=True)
# Seleccionar los últimos n valores
años_a_mostrar = 15
if dfA['Year'].max()-dfA['Year'].min()+1>años_a_mostrar:
    años_a_mostrar = años_a_mostrar
else:
    años_a_mostrar = dfA['Year'].max()-dfA['Year'].min()+1

dfA_summary = dfA_summary.tail(años_a_mostrar)

# Convertir la suma a entero
dfA_summary['Nº de  citas'] = dfA_summary['Nº de  citas'].apply(int)
latex_table = dfA_summary.to_latex()

# Imprimir el resultado
print(dfA_summary)
# Agregar la tabla al archivo informe.tex
caption = 'Nº de publicaciones y cantidad de citas de los últimos ' + str(años_a_mostrar) +' años.'
label = 'tab: nº publicaciones y citas'
latex_table = crear_tabla_latex(latex_table, caption, label)

agregar_tabla_latex(ruta_archivo, nombre_archivo_tex,latex_table)


      Nº artículos  Nº de  citas
Año                             
2009            15            94
2010            15            61
2011            13            73
2012            16            73
2013            15            71
2014            37           116
2015            15            54
2016            15            66
2017            15            28
2018            15            34
2019            15            12
2020            15            17
2021            15             1
2022             4             0


In [15]:

import plotly.express as px
dfA_plot = dfA.groupby('Year')['EID'].count().reset_index()
fig = px.bar(dfA_plot, x='Year', y='EID',template=template_elegido)
fig.update_layout(title='Nº de artículos en los últimos '+str(años_a_mostrar)+' años' )
fig.update_xaxes(range=[dfA['Year'].max()-años_a_mostrar, dfA['Year'].max()])
fig.update_yaxes(title='Nº de artículos')
fig.update_xaxes(title='Año')

fig.show()
nombre_archivo =abrev +'_'+ "_n_" +str(len(dfA))+'articulos_ultimos_'+str(años_a_mostrar)+'_anos_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)



lista_párrafo2=['De los ',str(len(dfA)),' documentos analizados en este informe, ',
                str(len(dfA[dfA['Year']>=dfA['Year'].max()-años_a_mostrar])),
                ' fueron realizados en los últimos ',str(años_a_mostrar), ' años, es decir, el',
                 str(round(100*len(dfA[dfA['Year']>=dfA['Year'].max()-años_a_mostrar])/len(dfA),1)),
                '\\% del total. Su distribución se muestra en la figura \\ref{fig:'+nombre_archivo+'}.'
                   
]

#
párrafo2=' '.join(lista_párrafo2)
print(párrafo2)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo2)




fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=300)
nombre_archivo_figura=nombre_archivo
leyenda='Artículos publicados en '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)


De los  220  documentos analizados en este informe,  220  fueron realizados en los últimos  14  años, es decir, el 100.0 \% del total. Su distribución se muestra en la figura \ref{fig:RELIME-RIITMA__n_220articulos_ultimos_14_anos_2023_1_12_12_55}.


In [16]:
import plotly.express as px

dfA_plot = dfA.groupby('Year')['Cited by'].sum().reset_index()
fig = px.bar(dfA_plot, x='Year', y='Cited by', template = template_elegido)
fig.update_layout(title='Nº de citas en los últimos '+str(años_a_mostrar)+' años' )
fig.update_xaxes(range=[dfA['Year'].max()-años_a_mostrar, dfA['Year'].max()])
fig.update_yaxes(title='Nº de citas')
fig.update_xaxes(title='Año')

fig.show()
nombre_archivo =abrev +'_'+ "_n_" +str(len(dfA))+'citas_ultimos_'+str(años_a_mostrar)+'_anos_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)




lista_párrafo3=['Los documentos tienen en total  ',str(dfA['Cited by'].sum().astype(int)),' citas. De las cuales,  ',
                str((dfA[dfA['Year']>=dfA['Year'].max()-años_a_mostrar]['Cited by'].sum()).astype(int)),
                ' fueron realizados en los últimos ',str(años_a_mostrar), ' años.',
                 'Su distribución se muestra en la figura \\ref{fig:'+nombre_archivo+'}.'
                   
]


párrafo3=' '.join(lista_párrafo3)
print(párrafo3)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo3)


fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=300)
nombre_archivo_figura=nombre_archivo
leyenda='Citas de artículos publicados en '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)
print(leyenda)



Los documentos tienen en total   700  citas. De las cuales,   700  fueron realizados en los últimos  14  años. Su distribución se muestra en la figura \ref{fig:RELIME-RIITMA__n_220citas_ultimos_14_anos_2023_1_12_12_55}.
Citas de artículos publicados en Scopus de RELIME-RIITMA. Fuente: Scopus.


# 5 Conteo de todos los campos antes de tratar la información

In [17]:
import plotly.express as px

dfK = dfA[dfA['Author Keywords'].notnull()]
dfK['key_words'] = dfK['Author Keywords'].apply(lambda x: x.split(';'))
dfK['key_words'] = dfK['key_words'].apply(lambda x: [word.strip() for word in x])

def minus(lista):
    return list(map(lambda x: x.lower(), lista))

dfK['key_words'] = dfK['key_words'].transform(minus)

key_word_list0 = sorted(dfK['key_words'].tolist())

#orden alfabético
from collections import Counter

key_word_count0 = dict(Counter(i for sub in key_word_list0 for i in set(sub)))
key_word_count0.pop('nan', None)

#ordenado
key_word_list0_ordenado = sorted(key_word_count0.items(), key=lambda x: x[1], reverse=True)


n = 50
key_word_list0_ordenado_elegidos = key_word_list0_ordenado[:n]

key_word_list0_ordenado_elegidos = key_word_list0_ordenado[:n]





##### gráfico palabras clave ###
title=str(n) + ' palabras clave más frecuentes de ' + abrev
fig=px.bar(template=template_elegido, orientation='h', y=[key_word_list0_ordenado_elegidos[j][0] for j in range(0,n-1)], x=[key_word_list0_ordenado_elegidos[j][1] for j in range(0,n-1)], text_auto=True)
fig.update_layout(
    title=title,
    yaxis_title="palabras clave",
    xaxis_title="frecuencia")
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=8)
fig.show()
nombre_archivo =abrev + "_" +str(n)+'_palabras_claves_mas_frecuentes_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=800)
nombre_archivo_figura=nombre_archivo




entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Palabras clave de la base de datos')


lista_párrafo4=['En la figura \\ref{fig:'+  nombre_archivo+'}',
                'se observan las ',str(n), 'palabras clave con mayor frecuencia en la',
                'base de datos analizada. La palabra que más aparece es <<',key_word_list0_ordenado_elegidos[0][0],'>>,',
                'la cual, se repite ', str(key_word_list0_ordenado_elegidos[0][1]), 'veces,',
                'es decir, en ', str(round(100*key_word_list0_ordenado_elegidos[0][1]/len(dfA),1)),
                '\% de los documentos.'
]


párrafo4=' '.join(lista_párrafo4)
print(párrafo4)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo4)



leyenda='Palabras clave proporcionadas por los autores en cada uno de los artículos. Fuente: '+fuente_artículos+'.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)
print(leyenda)

print('Solo se muestran las palabras clave más frecuentes porque en el título y abstract el' + '\n' +
'el proceso implica transformarlos')
for j in range(0,n-1):
    print(key_word_list0_ordenado_elegidos[j][0],': ',key_word_list0_ordenado_elegidos[j][1])

lista_palabras_clave=[key_word_list0_ordenado_elegidos[j][0]+': '+str(key_word_list0_ordenado_elegidos[j][1]) for j in range(0,n-1)]    
#entorno_enumerate_tex(ruta_archivo, nombre_archivo_tex,lista_palabras_clave)
print(len(key_word_list0_ordenado_elegidos[0][0]))
print((key_word_list0_ordenado_elegidos[0][1]))
print(lista_palabras_clave)

En la figura \ref{fig:RELIME-RIITMA_50_palabras_claves_mas_frecuentes_2023_1_12_12_55} se observan las  50 palabras clave con mayor frecuencia en la base de datos analizada. La palabra que más aparece es << mathematics >>, la cual, se repite  13 veces, es decir, en  5.9 \% de los documentos.
Palabras clave proporcionadas por los autores en cada uno de los artículos. Fuente: Scopus.
Solo se muestran las palabras clave más frecuentes porque en el título y abstract el
el proceso implica transformarlos
mathematics :  13
problem solving :  10
mathematics education :  10
modeling :  6
strategies :  5
apos theory :  5
socioepistemology :  5
secondary education :  5
mathematical working space :  5
teacher training :  5
teaching and learning :  4
geometrical working space :  4
mathematical activity :  4
professional development :  4
teaching :  4
teacher education :  4
visualization :  4
ethnomathematics :  3
mental calculation :  3
mathematics learning :  3
teaching experiment :  3
mathematica

# 6 Importar librerías sobre tratamiento del lenguaje natural

In [18]:
# Importación de nltk, lo oculté
import nltk
nltk.download('popular')

import re
import string

#lo saqué de acá: https://code-examples.net/es/q/152ebc5

wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()


stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('The')
stopwords.append('use')



[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/jorgegaona/nltk_data...
[nltk_data]    |   Package 

# 7  Limpieza de conectores e ilativos

## 7.1 Definición de funciones para la limpieza

In [19]:
def clean_text(txt):
    txt="".join([c for c in txt if c not in string.punctuation])
    tokens = re.split('\W+',txt)
    txt = [word for word in tokens if word not in stopwords]
    return txt

lista_eliminar = ['al', 'ano', 'año', 'com', 'commun', 'como', 'con', 'da', 'de', 'del', 'desd', 'do', 'e', 'el', 'em', 'en','et', 'entr', 'la','las', 'lo','los', 'na', 'para', 'por', 'que','sobre', 'sobr', 'su', 'sua', 'toward', 'two','um', 'uma', 'un', 'una', 'use', 'uso']

def eliminar_palabras(lista):
    for x in range(0,len(lista_eliminar)):
        if lista_eliminar[x] in lista:
            lista.remove(lista_eliminar[x])            
    return(lista)



## 7.2 Limpieza de palabras clave

In [20]:
dfA['Author Keywords']=dfA['Author Keywords'].astype(str)
dfA['Author Keywords']=dfA['Author Keywords'].str.lower()
dfA['key_words_non_spot'] = dfA['Author Keywords'].apply(lambda x : clean_text(x))

## 7.3 Limpieza de palabras de títulos

In [21]:
dfA['Title']=dfA['Title'].astype(str)
dfA['Title']=dfA['Title'].str.lower()
dfA['title_non_spot'] = dfA['Title'].apply(lambda x : clean_text(x))

## 7.4 Limpieza de palabras de resúmenes


In [22]:
dfA['Abstract']=dfA['Abstract'].astype(str)
dfA['Abstract']=dfA['Abstract'].str.lower()
dfA['abstract_non_spot'] = dfA['Abstract'].apply(lambda x : clean_text(x))

# 8 Aplicar método lemmatize o stemming

## 8.1 Definición del método

In [23]:
def lemmatization(token_txt):
    text=[wn.lemmatize(word) for word in token_txt]
    return text

def stemming(token_txt):
    text=[ps.stem(word) for word in token_txt]
    return text

      


    

## 8.2 Aplicación del método sobre todos los campos

In [24]:
if método == 1:
    met = 'lemmatize'
    dfA['key_words_metodo'] = dfA['key_words_non_spot'].apply(lambda x: lemmatization(x))
    dfA['key_words_metodo'] = dfA['key_words_metodo'].apply(lambda x: list(set(x)))
    dfA['key_words_metodo'] = dfA['key_words_metodo'].apply(lambda x: eliminar_palabras(x))
    
    dfA['title_metodo']=dfA['title_non_spot'].apply(lambda x: lemmatization(x))
    dfA['title_metodo'] = dfA['title_metodo'].apply(lambda x: list(set(x)))
    dfA['title_metodo'] = dfA['title_metodo'].apply(lambda x: eliminar_palabras(x))
    
    dfA['abstract_metodo']=dfA['abstract_non_spot'].apply(lambda x: lemmatization(x))
    dfA['abstract_metodo'] = dfA['abstract_metodo'].apply(lambda x: list(set(x)))
    dfA['abstract_metodo'] = dfA['abstract_metodo'].apply(lambda x: eliminar_palabras(x))
else:
    met = 'stemming'
    dfA['key_words_metodo'] = dfA['key_words_non_spot'].apply(lambda x: stemming(x))
    dfA['key_words_metodo'] = dfA['key_words_metodo'].apply(lambda x: list(set(x)))
    dfA['key_words_metodo'] = dfA['key_words_metodo'].apply(lambda x: eliminar_palabras(x))
    
    dfA['title_metodo']=dfA['title_non_spot'].apply(lambda x: stemming(x))
    dfA['title_metodo'] = dfA['title_metodo'].apply(lambda x: list(set(x)))
    dfA['title_metodo'] = dfA['title_metodo'].apply(lambda x: eliminar_palabras(x))
    
    dfA['abstract_metodo']=dfA['abstract_non_spot'].apply(lambda x: stemming(x))
    dfA['abstract_metodo'] = dfA['abstract_metodo'].apply(lambda x: list(set(x)))
    dfA['abstract_metodo'] = dfA['abstract_metodo'].apply(lambda x: eliminar_palabras(x))
   
 
def combinar_title_key(row):
    return row['title_metodo'] + row['key_words_metodo']

def combinar_title_abs(row):
    return row['title_metodo'] + row['abstract_metodo']

def combinar_key_abs(row):
    return row['key_words_metodo'] + row['abstract_metodo']
def combinar_title_abs_key(row):
    return row['title_metodo'] +  row['abstract_metodo'] + row['key_words_metodo'] 


# campo 4
dfA['title_key_metodo'] = dfA.apply(combinar_title_key, axis=1)
dfA['title_key_metodo'] = dfA['title_key_metodo'].apply(lambda x: list(set(x)))

# campo 5
dfA['title_abs_metodo'] = dfA.apply(combinar_title_abs, axis=1)
dfA['title_abs_metodo'] = dfA['title_abs_metodo'].apply(lambda x: list(set(x)))

# campo 6
dfA['key_abs_metodo'] = dfA.apply(combinar_key_abs, axis=1)
dfA['key_abs_metodo'] = dfA['key_abs_metodo'].apply(lambda x: list(set(x)))

# campo 7
dfA['title_abs_key_metodo'] = dfA.apply(combinar_title_abs_key, axis=1)
dfA['title_abs_key_metodo'] = dfA['title_abs_key_metodo'].apply(lambda x: list(set(x)))

lista_campos = ['key_words_metodo','title_metodo','abstract_metodo','title_key_metodo','title_abs_metodo','key_abs_metodo','title_abs_key_metodo']

nombre_campos = ['PALABRAS CLAVE','TÍTULOS','RESÚMENES','TÍTULO - PALABRAS CLAVE','TÍTULO - RESÚMENES','PALABRAS CLAVE - RESÚMENES','TÍTULO - PALABRAS CLAVE - RESÚMENES']

dic_campo_a_contar_palabras = dict(zip(dic_campo_a_analizar.values(), lista_campos))
dic_nombres_campo = dict(zip(dic_campo_a_analizar.values(), nombre_campos))


campo_a_analizar

print(dic_campo_a_contar_palabras)
print(dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))
print(dic_campo_a_analizar.get(campo_a_analizar))


{'key_words': 'key_words_metodo', 'title': 'title_metodo', 'abstract': 'abstract_metodo', 'title_key': 'title_key_metodo', 'title_abs': 'title_abs_metodo', 'key_abs': 'key_abs_metodo', 'title_abs_key': 'title_abs_key_metodo'}
PALABRAS CLAVE
key_words


## 8.4 Conteo y gráfica del método aplicado al campo elegido

In [25]:


campo_a_contar_palabras = dic_campo_a_contar_palabras.get(dic_campo_a_analizar.get(campo_a_analizar))

campo_con_palabras = sorted(dfA[campo_a_contar_palabras].tolist())

cuenta_campo_a_analizar = dict(Counter(i for sub in campo_con_palabras for i in set(sub)))
cuenta_campo_a_analizar.pop('nan', None)

cuenta_campo_a_analizar_ordenado = sorted(cuenta_campo_a_analizar.items(), key=lambda x: x[1], reverse=True)

cuenta_campo_a_analizar_ordenado_elegidos = cuenta_campo_a_analizar_ordenado[:n]

cuenta_campo_a_analizar_ordenado_elegidos
title= str(n) + ' palabras más frecuentes de '  + abrev + ' en : ' + dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar))

fig=px.bar(template=template_elegido,  orientation='h', y=[cuenta_campo_a_analizar_ordenado_elegidos[j][0] for j in range(0,n-1)], x=[cuenta_campo_a_analizar_ordenado_elegidos[j][1] for j in range(0,n-1)], text_auto=True)
fig.update_layout(
    title=title,
    yaxis_title="palabras más frecuentes",
    xaxis_title="frecuencia")
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=8)

fig.show()
nombre_archivo =abrev +'_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=800)





# 9 comienzo GRAFO método y campo elegido

## 9.1 Definición de parejas

In [26]:
def parejas(lista):
    salida1 = [(lista[x],lista[y]) for x in range(0,len(lista)) for y in range(0,len(lista)) if y>x]
    return salida1

#dic_campo_a_analizar = {1:'key_words',2:'title',3:'abstract',4:'title_key',5:'title_abs',6:'key_abs',7:'title_abs_key'}


if campo_a_analizar == 1:
    dfA['parejas_key_words'] = dfA['key_words_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
else:
    if campo_a_analizar == 2 :
        dfA['parejas_title'] = dfA['title_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
    else:
        if campo_a_analizar == 3 :
            dfA['parejas_abstract'] = dfA['abstract_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
        else:
            if campo_a_analizar == 4 :
                dfA['parejas_title_key'] = dfA['title_key_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
            else:
                if campo_a_analizar == 5 :
                    dfA['parejas_title_abs'] = dfA['title_abs_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
                else:
                    if campo_a_analizar == 6 :
                        dfA['parejas_key_abs'] = dfA['key_abs_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
                    else:
                        dfA['parejas_title_abs_key'] = dfA['title_abs_key_metodo'].apply(lambda x: sorted(list(set(x)))).transform(parejas)
                        

#dfA.to_excel('datos_parejas.xlsx')     

        


## 9.2  definición de campo para nodos y aristas 

In [27]:
!pip install pyvis

In [28]:
if campo_a_analizar == 1:
    campo_para_nodos_grafo = 'key_words_metodo'
    campo_para_aristas_grafo = 'parejas_key_words'
else:
    if campo_a_analizar == 2 :
        campo_para_nodos_grafo = 'title_metodo'
        campo_para_aristas_grafo = 'parejas_title'
    else:
        if campo_a_analizar == 3 :
            campo_para_nodos_grafo = 'abstract_metodo'
            campo_para_aristas_grafo = 'parejas_abstract'
        else:
            if campo_a_analizar == 4 :
                campo_para_nodos_grafo = 'title_key_metodo'
                campo_para_aristas_grafo = 'parejas_title_key'
            else:
                if campo_a_analizar == 5 :
                    campo_para_nodos_grafo = 'title_abs_metodo'
                    campo_para_aristas_grafo = 'parejas_title_abs'
                else:
                    if campo_a_analizar == 6 :
                        campo_para_nodos_grafo = 'key_abs_metodo'
                        campo_para_aristas_grafo = 'parejas_key_abs'
                    else:
                        campo_para_nodos_grafo = 'title_abs_key_metodo'
                        campo_para_aristas_grafo = 'parejas_title_abs_key'
                        


                        
print(campo_para_nodos_grafo)
print(campo_para_aristas_grafo) 


key_words_metodo
parejas_key_words


## 9.2 Definición de grafo

In [29]:
import time
inicio = time.time()

#!pip install pyvis
from pyvis.network import Network
import networkx as nx


df1=dfA

G = nx.Graph()
lista_nodos=[]
lista_aristas=[]



for x in range(0,len(df1)):
    lista_nodos.extend(df1.iloc[x][campo_para_nodos_grafo])
    lista_aristas.extend(df1.iloc[x][campo_para_aristas_grafo])


cuenta_pesos_nodos = {x:lista_nodos.count(x) for x in lista_nodos}
cuenta_pesos_nodos = dict(sorted(cuenta_pesos_nodos.items(), key=lambda item:item[1],reverse=True))

cuenta_pesos_aristas = {x:lista_aristas.count(x) for x in lista_aristas}
cuenta_pesos_aristas = dict(sorted(cuenta_pesos_aristas.items(), key=lambda item:item[1],reverse=True))

#creación de nodos de G
for clave, valor in cuenta_pesos_nodos.items():
    G.add_node(clave,label=clave+' ('+str(valor)+')',value=(valor))

#creación de aristas de G    
for clave, valor in cuenta_pesos_aristas.items():
    G.add_edge(clave[0],clave[1], value = (valor),label=str(valor)) 
    
print('Información del grafo:')
print('Grafo de la revista: ',abrev)
print('Cantidad de artículos: ',len(df1))
print('Cantidad de nodos grafo completo G: ',(len(G.nodes)))
print('Cantidad de aristas grafo completo G: ',(len(G.edges)))
print(dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))

fin = time.time()
print(fin-inicio, ' seg. Equivalentes a ',(fin-inicio)/60, ' min') 


Información del grafo:
Grafo de la revista:  RELIME-RIITMA
Cantidad de artículos:  220
Cantidad de nodos grafo completo G:  578
Cantidad de aristas grafo completo G:  4713
PALABRAS CLAVE
0.726830005645752  seg. Equivalentes a  0.012113833427429199  min


### 9.2.1 Informe Latex:  Espacio rizomático conceptual (ERC)

In [30]:
entorno_nueva_página_tex(ruta_archivo, nombre_archivo_tex)
entorno_section_tex(ruta_archivo, nombre_archivo_tex, 'Espacio rizomático conceptual (ERC) a partir de las palabras en: '+dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))
entorno_sub_section_tex(ruta_archivo, nombre_archivo, 'Información general del ERC')



parrafo_tex(ruta_archivo, nombre_archivo_tex, 'La información general del ERC es la siguiente:')


lista_información_general=['Grafo de la base de datos denominada: ' + abrev,
    'El método utilizado fue: '+met,
    'Cantidad de artículos: ' + str(len(df1)),
    'Cantidad de nodos grafo completo G: '+ str(len(G.nodes)),
    'Cantidad de aristas grafo completo G: '+str(len(G.edges)),
    'Tiempo requerido para generar el grafo: '+ str(round(fin-inicio,1))+ ' seg. Equivalentes a '+ str(round((fin-inicio)/60,1))+ ' min.'
]

entorno_itemize_tex(ruta_archivo, nombre_archivo_tex,lista_información_general)
print(lista_información_general)

print('Espacio rizomático conceptual (ERC) a partir de las palabras en: '+dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))

['Grafo de la base de datos denominada: RELIME-RIITMA', 'El método utilizado fue: lemmatize', 'Cantidad de artículos: 220', 'Cantidad de nodos grafo completo G: 578', 'Cantidad de aristas grafo completo G: 4713', 'Tiempo requerido para generar el grafo: 0.7 seg. Equivalentes a 0.0 min.']
Espacio rizomático conceptual (ERC) a partir de las palabras en: PALABRAS CLAVE


## 9.3 Calcular cantidad de nodos

In [31]:
cantidad_aristas_adecuadas=100
cantidad_nodos_adecuados=100


nodos= []
cantidad_nodos = []

for i in range(1,100):
    nodos.append(i)
    (cantidad_nodos.append(len([n for n,v in G.nodes(data=True) if v['value'] > i ])))

import plotly.express as px
title='Cantidad de nodos según el peso mínimo del nodo'
fig = px.scatter(x=nodos, y=cantidad_nodos, title=title)
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de los nodos",
    yaxis_title="cantidad de nodos")
fig.show()
nombre_archivo =abrev + "_cantidad de nodos " +str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf", height=310,width=550)



title='Cantidad de nodos según el peso mínimo del nodo (zoom)'
fig = px.scatter(x=nodos, y=cantidad_nodos, title=title)
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de los nodos",
    yaxis_title="cantidad de nodos")
fig.update_yaxes(range=[0, 200])
fig.show()   
nombre_archivo =abrev + "_cantidad_de_nodos_zoom_" +str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)





fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf", height=310,width=550)


pares_ordenados_nodos=[[nodos[i],cantidad_nodos[i]] for i in range(0,len(nodos)) if cantidad_nodos[i]<=cantidad_nodos_adecuados]




tamaño_nodo = pares_ordenados_nodos[0][0]


print(tamaño_nodo)

print(cantidad_nodos)
print(pares_ordenados_nodos)



3
[212, 133, 81, 62, 51, 44, 38, 30, 24, 21, 17, 15, 13, 12, 12, 11, 9, 8, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 4, 4, 4, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[3, 81], [4, 62], [5, 51], [6, 44], [7, 38], [8, 30], [9, 24], [10, 21], [11, 17], [12, 15], [13, 13], [14, 12], [15, 12], [16, 11], [17, 9], [18, 8], [19, 7], [20, 6], [21, 6], [22, 6], [23, 6], [24, 6], [25, 6], [26, 6], [27, 6], [28, 6], [29, 6], [30, 6], [31, 5], [32, 5], [33, 4], [34, 4], [35, 4], [36, 3], [37, 3], [38, 3], [39, 2], [40, 1], [41, 1], [42, 1], [43, 1], [44, 1], [45, 1], [46, 1], [47, 1], [48, 1], [49, 1], [50, 1], [51, 1], [52, 1], [53, 1], [54, 0], [55, 0], [56, 0], [57, 0], [58, 0], [59, 0], [60, 0], [61, 0], [62, 0], [63, 0], [64, 0], [65, 0], [66, 0], [67, 0], [68, 0], [69, 0], [70, 0], [71, 0], [72, 0], [73, 0], [74, 0], [75, 0], [76, 0], [77, 0],

### 9.3.1 Informe Latex:  Calcular cantidad de nodos

In [32]:
entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Nodos')


lista_información_nodos=['Como se indicó en la información general,',
                         'el ERC tiene '+ str(len(G.nodes)), 'nodos.',
    'Para que en el ERC se puedan distinguir los nodos más importantes,', 
    'en la figura \\ref{fig:'+nombre_archivo+'} se muestra un gráfico con ',
    'la cantidad de nodos en función del tamaño de los mismos.',
    'Se estableció como una cantidad adecuada para visualizar el ERC, ',
    str(cantidad_nodos_adecuados), ' nodos. Esto da como resultado, que el ERC mostrará',
    'los nodos que tengan un tamaño igual o superior a '+str(tamaño_nodo)+'. Es decir, si',
    'una palabra se repite al menos '+str(tamaño_nodo),'veces en los ',str(len(dfA)),
    'documentos, entonces aparecerá en el Espacio Rizomático Conceptual de '+abrev+'.'
]
párrafo=' '.join(lista_información_nodos)
print(párrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

nombre_archivo_figura=nombre_archivo
leyenda='Cantidad de nodos del ERC en función del tamaño de los nodos. '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'. Elaboración propia.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)

print(leyenda)
print(tamaño_nodo)



Como se indicó en la información general, el ERC tiene 578 nodos. Para que en el ERC se puedan distinguir los nodos más importantes, en la figura \ref{fig:RELIME-RIITMA_cantidad_de_nodos_zoom_2023_1_12_12_55} se muestra un gráfico con  la cantidad de nodos en función del tamaño de los mismos. Se estableció como una cantidad adecuada para visualizar el ERC,  100  nodos. Esto da como resultado, que el ERC mostrará los nodos que tengan un tamaño igual o superior a 3. Es decir, si una palabra se repite al menos 3 veces en los  220 documentos, entonces aparecerá en el Espacio Rizomático Conceptual de RELIME-RIITMA.
Cantidad de nodos del ERC en función del tamaño de los nodos. Scopus de RELIME-RIITMA. Fuente: Scopus. Elaboración propia.
3


In [33]:


lista_párrafo_nodos=['Las ' +str(n)+' palabras que más aparecen en ', dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)),
                     'son las que se muestran en la figura \\ref{fig:',nombre_archivo+'}.',
                     'Los resultados  muestran que las 3 las palabras más frecuentes son: ',
                cuenta_campo_a_analizar_ordenado_elegidos[0][0]+',', cuenta_campo_a_analizar_ordenado_elegidos[1][0],
                ' y ', cuenta_campo_a_analizar_ordenado_elegidos[2][0]+', con', 
                str(cuenta_campo_a_analizar_ordenado_elegidos[0][1])+' ('+str(round(100*cuenta_campo_a_analizar_ordenado_elegidos[0][1]/len(dfA),1))+'\%),',
                str(cuenta_campo_a_analizar_ordenado_elegidos[1][1])+' ('+str(round(100*cuenta_campo_a_analizar_ordenado_elegidos[1][1]/len(dfA),1))+'\%),',
                ' y ',str(cuenta_campo_a_analizar_ordenado_elegidos[2][1])+' ('+str(round(100*cuenta_campo_a_analizar_ordenado_elegidos[2][1]/len(dfA),1))+'\%)'
                ' repeticiones respectivamente.'
                ]


párrafo=' '.join(lista_párrafo_nodos)
print(párrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

#nombre_archivo_figura=nombre_archivo
#leyenda='Análisis de las ' + str(n)+ ' palabras más frecuentes en '+dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar))+  ', de la base de datos ' + abrev+ ', luego de separar, limpiar y aplicar el método '+ met+ ' de nltk al campo elegido.'
#entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)
print('\n')
print(leyenda)

Las 50 palabras que más aparecen en  PALABRAS CLAVE son las que se muestran en la figura \ref{fig: RELIME-RIITMA_cantidad_de_nodos_zoom_2023_1_12_12_55}. Los resultados  muestran que las 3 las palabras más frecuentes son:  mathematics, education  y  teacher, con 54 (24.5\%), 40 (18.2\%),  y  36 (16.4\%) repeticiones respectivamente.


Cantidad de nodos del ERC en función del tamaño de los nodos. Scopus de RELIME-RIITMA. Fuente: Scopus. Elaboración propia.


## 9.4 Calcular cantidad de los aristas

In [34]:
aristas= []
cantidad_aristas = []
for j in range(1,50):
    aristas.append(j)
    cantidad_aristas.append(len(dict(filter(lambda x: x[1] >= j, cuenta_pesos_aristas.items()))))    
    #print(j,',',len(dict(filter(lambda x: x[1] >= j, cuenta_pesos_aristas.items()))))
    
import plotly.express as px

title='Cantidad de aristas  según peso mínimo de las aristas'

fig = px.scatter(x=aristas, y=cantidad_aristas, title=title)
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de las aristas",
    yaxis_title="cantidad de aristas")

fig.show()
nombre_archivo =abrev + "_cantidad de aristas " +str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",height=310,width=550)

title='Cantidad de aristas según peso mínimo de las aristas (zoom)'
fig = px.scatter(x=aristas, y=cantidad_aristas, title=title)
fig.update_yaxes(range=[0, 200])
fig.update_layout(
    title=title,
    xaxis_title="peso mínimo de las aristas",
    yaxis_title="cantidad de aristas")

fig.show()
nombre_archivo =abrev + "_cantidad_de_aristas_zoom_" +str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf", height=310,width=550)

pares_ordenados_aristas=[[aristas[i],cantidad_aristas[i]] for i in range(0,len(aristas)) if cantidad_aristas[i]<=cantidad_aristas_adecuadas]
filtro_arista = pares_ordenados_aristas[0][0]
print(filtro_arista)
print(pares_ordenados_aristas)



4
[[4, 72], [5, 47], [6, 31], [7, 18], [8, 13], [9, 11], [10, 10], [11, 8], [12, 7], [13, 6], [14, 5], [15, 4], [16, 3], [17, 2], [18, 1], [19, 1], [20, 1], [21, 1], [22, 1], [23, 1], [24, 1], [25, 1], [26, 1], [27, 1], [28, 1], [29, 1], [30, 0], [31, 0], [32, 0], [33, 0], [34, 0], [35, 0], [36, 0], [37, 0], [38, 0], [39, 0], [40, 0], [41, 0], [42, 0], [43, 0], [44, 0], [45, 0], [46, 0], [47, 0], [48, 0], [49, 0]]


### 9.4.1 Informe Latex:  Calcular cantidad de aristas


In [35]:
entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Aristas')


lista_información_aristas=['Por otra parte, el ERC tiene '+ str(len(G.edges)), 'aristas.',
    'Para que en el ERC se puedan distinguir las aristas más importantes,', 
    'en la figura \\ref{fig:'+nombre_archivo+'} se muestra un gráfico con ',
    'la cantidad de aristas en función de su tamaño.',
    'Se estableció como una cantidad adecuada para visualizar el ERC, ',
    str(cantidad_aristas_adecuadas), ' aristas. Esto da como resultado, que el ERC mostrará',
    'las aristas que tengan un tamaño igual o superior a '+str(filtro_arista)+'. Es decir, si',
    'la relación entre dos palabras palabra se repite al menos '+str(filtro_arista),'veces en los ',str(len(dfA)),
    'documentos, entonces aparecerá en el Espacio Rizomático Conceptual de '+abrev+'.'
]
párrafo=' '.join(lista_información_aristas)
print(párrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

nombre_archivo_figura=nombre_archivo
leyenda='Cantidad de aristas del ERC en función del tamaño de las aristas. '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'. Elaboración propia.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)


                         

párrafo=' '.join(lista_información_aristas)
print(párrafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

print(leyenda)


Por otra parte, el ERC tiene 4713 aristas. Para que en el ERC se puedan distinguir las aristas más importantes, en la figura \ref{fig:RELIME-RIITMA_cantidad_de_aristas_zoom_2023_1_12_12_55} se muestra un gráfico con  la cantidad de aristas en función de su tamaño. Se estableció como una cantidad adecuada para visualizar el ERC,  100  aristas. Esto da como resultado, que el ERC mostrará las aristas que tengan un tamaño igual o superior a 4. Es decir, si la relación entre dos palabras palabra se repite al menos 4 veces en los  220 documentos, entonces aparecerá en el Espacio Rizomático Conceptual de RELIME-RIITMA.
Por otra parte, el ERC tiene 4713 aristas. Para que en el ERC se puedan distinguir las aristas más importantes, en la figura \ref{fig:RELIME-RIITMA_cantidad_de_aristas_zoom_2023_1_12_12_55} se muestra un gráfico con  la cantidad de aristas en función de su tamaño. Se estableció como una cantidad adecuada para visualizar el ERC,  100  aristas. Esto da como resultado, que el ERC 

## 9.4 Definición manual del tamaño de nodos y aristas

In [36]:
#Cambio de cantidad de nodos y aristas manual
#hay que borrar los símbolos # que están delante de las dos líneas siguientes

#tamaño_nodo = 2
#filtro_arista = 3


## 9.5 Grafo con filtros de nodo y aristas

In [37]:
inicio = time.time()



cuenta_pesos_aristas_H=dict(filter(lambda x: x[1] >= filtro_arista, cuenta_pesos_aristas.items()))

H = nx.Graph()

#creación de nodos de H
for clave, valor in cuenta_pesos_nodos.items():
    #print(clave,valor)
    H.add_node(clave,label=clave+' ('+str(valor)+')',value=(valor))

#creación de aristas de H    
for clave, valor in cuenta_pesos_aristas_H.items():
    #print(clave[0],',',valor)
    H.add_edge(clave[0],clave[1], value = (valor),label=str(valor)) 

selected_nodes = [n for n,v in G.nodes(data=True) if v['value'] >= tamaño_nodo ]
H = H.subgraph(selected_nodes)

fin = time.time()

print(abrev)
print(fin-inicio, ' seg. Equivalentes a ',(fin-inicio)/60, ' min') 
print('\n')
print('Cantidad de nodos subgrafo completo H: ',(len(H.nodes)))
print('Cantidad de aristas subgrafo completo H: ',(len(H.edges)))

print('\n')
print('Nodos de tamaño como mínimo: ', tamaño_nodo)
print('Aristas de tamaño como mínimo: ', filtro_arista)

inicio = time.time()
net=Network(notebook=True)
net.repulsion()
net.from_nx(H)
nombre_archivo = 'grafo_'+ abrev + '_' + campo_para_nodos_grafo + '_met_'+met + '_'+'_n_'+str(len(df1.index))+'_''_f_nodo_'+str(tamaño_nodo)+'_f_edge_'+str(filtro_arista)

net.show(abrev + '/'+nombre_archivo+'.html')


print('\caption{','Extracto del Espacio Rizomático Conceptual' ' para ',abrev, 'que incluye' ,len(dfA), 
      ' artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos ',str(tamaño_nodo),
      ' y las aristas de al menos tamaño ',filtro_arista,'.}')


print(nombre_archivo)

RELIME-RIITMA
0.0017271041870117188  seg. Equivalentes a  2.8785069783528646e-05  min


Cantidad de nodos subgrafo completo H:  133
Cantidad de aristas subgrafo completo H:  72


Nodos de tamaño como mínimo:  3
Aristas de tamaño como mínimo:  4
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
\caption{ Extracto del Espacio Rizomático Conceptual para  RELIME-RIITMA que incluye 220  artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos  3  y las aristas de al menos tamaño  4 .}
grafo_RELIME-RIITMA_key_words_metodo_met_lemmatize__n_220__f_nodo_3_f_edge_4


In [38]:
parrafo_tex(ruta_archivo, nombre_archivo_tex, "\\newpage \n'")

entorno_sub_section_tex(ruta_archivo, nombre_archivo_tex, 'Nodos y aristas más frecuentes')
nombre_archivo =abrev +'_nodos_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)

lista_información_grafo=['Por lo tanto ',
r"$\mathcal{G}_{\eta=",
str(tamaño_nodo),
r",\alpha=",str(filtro_arista),
r"}=(V_{\eta=",str(tamaño_nodo),
r",\alpha=",str(filtro_arista),r"},E_{\eta=",  
str(tamaño_nodo),r",\alpha=",str(filtro_arista),'})$',  
'es un grafo tal que, los nodos tienen tamaño mayor o igual a ', 
str(tamaño_nodo),' y las aristas tienen tamaño mayor o igual a ',
str(filtro_arista)+'.',' Esto produce un grafo que tiene ',str(len(H.nodes)),
'nodos y ', str(len(H.edges)), 'aristas. ','\n',
'En la figura \\ref{fig:'+nombre_archivo+'} se muestra un gráfico con los 25 nodos con mayor frecuencia.'                         
                         ]

párrafo=' '.join(lista_información_grafo)
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)

lista_información_grafo=['Los ', str(n), 'nodos más importantes se muestran '
                        ]                         
print(párrafo)



campo_a_contar_palabras = dic_campo_a_contar_palabras.get(dic_campo_a_analizar.get(campo_a_analizar))

campo_con_palabras = sorted(dfA[campo_a_contar_palabras].tolist())

cuenta_campo_a_analizar = dict(Counter(i for sub in campo_con_palabras for i in set(sub)))
cuenta_campo_a_analizar.pop('nan', None)

cuenta_campo_a_analizar_ordenado = sorted(cuenta_campo_a_analizar.items(), key=lambda x: x[1], reverse=True)

cuenta_campo_a_analizar_ordenado_elegidos = cuenta_campo_a_analizar_ordenado[:n]

cuenta_campo_a_analizar_ordenado_elegidos
title= str(n) + ' nodos más frecuentes de '  + abrev + ' en : ' + dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar))

fig=px.bar(template=template_elegido, orientation='h', y=[cuenta_campo_a_analizar_ordenado_elegidos[j][0] for j in range(0,n-1)], x=[cuenta_campo_a_analizar_ordenado_elegidos[j][1] for j in range(0,n-1)], text_auto=True)
fig.update_layout(
    title=title,
    yaxis_title="nodos más frecuentes",
    xaxis_title="frecuencia")
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=8)

fig.show()
nombre_archivo =abrev +'_nodos_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)
fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=800)

nombre_archivo_figura=nombre_archivo
leyenda='Artículos publicados en '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)

párrafo = 'En la figura \\ref{fig:'+nombre_archivo+'} se muestra un gráfico con las 25 aristas con mayor frecuencia.'
parrafo_tex(ruta_archivo, nombre_archivo_tex, párrafo)



Por lo tanto  $\mathcal{G}_{\eta= 3 ,\alpha= 4 }=(V_{\eta= 3 ,\alpha= 4 },E_{\eta= 3 ,\alpha= 4 })$ es un grafo tal que, los nodos tienen tamaño mayor o igual a  3  y las aristas tienen tamaño mayor o igual a  4.  Esto produce un grafo que tiene  133 nodos y  72 aristas.  
 En la figura \ref{fig:RELIME-RIITMA_nodos_key_words_n_50_met_lemmatize_2023_1_12_12_55} se muestra un gráfico con los 25 nodos con mayor frecuencia.


In [39]:

lista_aristas_H_ordenada = sorted(H.edges(data=True), key=lambda x: x[2]['value'],reverse=True)

lista_aristas_H=[]
lista_valores_aristas_H=[]
for x in lista_aristas_H_ordenada:
    lista_aristas_H.append(x[0]+'-'+x[1])
    lista_valores_aristas_H.append(x[2].get('value'))
    #print(x[0],'-',x[1],':',x[2].get('value'))

title= str(n) + ' aristas  más frecuentes de '  + abrev + ' en : ' + dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar))

fig=px.bar(template=template_elegido, orientation='h', y=[lista_aristas_H[:n][j] for j in range(0,n-1)], x=[lista_valores_aristas_H[:n][j] for j in range(0,n-1)], text_auto=True)
fig.update_layout(
    title=title,
    xaxis_title="aristas más frecuentes",
    yaxis_title="frecuencia")
fig.update_xaxes(tickangle=35)
fig.update_layout(font_size=8)

fig.show()
nombre_archivo =abrev +'_aristas_'+dic_campo_a_analizar.get(campo_a_analizar)+ "_n_" +str(n)+'_met_'+met+'_'+str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)

fig.write_html(abrev + '/' + nombre_archivo+".html")
fig.write_image(abrev + '/' + nombre_archivo+".pdf",width=600, height=800)


nombre_archivo_figura=nombre_archivo
leyenda='Artículos publicados en '+fuente_artículos+  ' de '+abrev+'. Fuente: '+fuente_artículos+'.'
entorno_figura_tex(ruta_archivo, nombre_archivo_tex,nombre_archivo_figura, leyenda)


print(lista_aristas_H)
print(lista_valores_aristas_H)

['education-mathematics', 'teacher-mathematics', 'space-working', 'education-teacher', 'mathematics-teaching', 'solving-problem', 'teacher-training', 'learning-mathematics', 'knowledge-teacher', 'teacher-teaching', 'education-teaching', 'space-mathematical', 'mathematical-working', 'learning-teaching', 'geometry-teaching', 'geometrical-space', 'knowledge-mathematics', 'mathematical-modeling', 'learning-education', 'geometry-dynamic', 'geometrical-working', 'mathematical-teacher', 'mathematical-problem', 'mathematical-activity', 'mathematical-education', 'mathematical-mathematics', 'professional-teacher', 'history-mathematics', 'preservice-education', 'preservice-teacher', 'education-secondary', 'development-professional', 'learning-teacher', 'solving-mathematical', 'geometry-mathematics', 'geometry-space', 'geometry-working', 'technology-mathematics', 'knowledge-mathematical', 'school-mathematics', 'school-teacher', 'mathematical-teaching', 'theory-apos', 'history-education', 'preservi

In [40]:

#lista_aristas_H_ordenada


### 9.5.1 Exportar a Datapane el grafo completo


In [41]:
dfA['Year'].unique()

array([2014, 2016, 2011, 2010, 2009, 2013, 2012, 2015, 2018, 2017, 2020,
       2019, 2021, 2022])

## 9.6 Podar el rizoma

In [42]:
lista_nodos= [e for e in G.nodes(data=True)][:10]
#print(lista_nodos)
filtro_arista = 10
tamaño_nodo = 10
cuenta_pesos_aristas_H=dict(filter(lambda x: x[1] >= filtro_arista, cuenta_pesos_aristas.items()))

H1 = nx.Graph()

#creación de nodos de H
for clave, valor in cuenta_pesos_nodos.items():
    #print(clave,valor)
    H1.add_node(clave,label=clave+' ('+str(valor)+')',value=(valor))

#creación de aristas de H    
for clave, valor in cuenta_pesos_aristas_H.items():
    H1.add_edge(clave[0],clave[1], value = (valor),label=str(valor)) 
palabras_a_podar=['education','nan']


fin = time.time()
net=Network(notebook=True)
net.repulsion()
net.from_nx(H1)
nombre_archivo = 'grafo_podado_'+ abrev + '_' + campo_para_nodos_grafo + '_met_'+ met +str(len(dfA)) +'_n_' + str(len(df1.index)) + '_f_nodo_'+ str(tamaño_nodo) + '_f_edge_'+ str(filtro_arista)
net.show(abrev + '/' + nombre_archivo+'.html')
print('----------------------------')
lista_nodosH1= [e for e in H1.nodes(data=True)]
lista_nodosH1_ordenada = sorted(lista_nodosH1, key=lambda x: x[1]['value'], reverse=True) # [:20]

caption_1= '\caption{','Extracto del Espacio Rizomático Conceptual'+' \textbf{podado} para ' + abrev + ' que incluye ' + str(len(dfA)) +  ' artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos ' + str(tamaño_nodo) + ' y las aristas de al menos tamaño ' + str(filtro_arista) +'. Las palabras podadas son: ' + ', '.join(palabras_a_podar[:len(palabras_a_podar)-1])+ ' y '+ palabras_a_podar[-1] +'.}'
 

print(caption_1)

print('filtro arista:', filtro_arista)
print('tamaño_nodo: ',tamaño_nodo)

print('\n')
print('Lista de aristas y pesos de ERC podado: ',)
#for x in lista_nodosH1_ordenada:
#    print(x[0],':',x[1].get('value'))

lista_aristas_H1_ordenada=[e for e in H1.edges(data=True)]
lista_aristas_H1_ordenada = sorted(lista_aristas_H1_ordenada, key=lambda x: x[2]['value'], reverse=True)



#for x in lista_aristas_H1_ordenada:
    #print(x[0],'-',x[1],':',x[2].get('value'))
    


Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
----------------------------
('\\caption{', 'Extracto del Espacio Rizomático Conceptual \textbf{podado} para RELIME-RIITMA que incluye 220 artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos 10 y las aristas de al menos tamaño 10. Las palabras podadas son: education y nan.}')
filtro arista: 10
tamaño_nodo:  10


Lista de aristas y pesos de ERC podado: 


### 9.6.1 Exportar a Datapane el grafo podado


import datapane as dp

with open(abrev + '/'+nombre_archivo+'.html', 'r') as f:
    html_network_1 = f.read()
    
dp.Report(
  dp.HTML(
    html_network_1
  )
).upload(name=abrev+' - espacio rizomático conceptual PODADO '+ str(len(dfA)) + ' artículos')

## 9.7 Ego de un grafo

In [43]:
ego='teacher'
tamaño_nodo = 5
filtro_arista = 8


cuenta_pesos_aristas_Ego=dict(filter(lambda x: x[1] >= filtro_arista, cuenta_pesos_aristas.items()))

Ego = nx.Graph()

#creación de nodos de H
for clave, valor in cuenta_pesos_nodos.items():
    #print(clave,valor)
    Ego.add_node(clave,label=clave+' ('+str(valor)+')',value=(valor))

#creación de aristas de H    
for clave, valor in cuenta_pesos_aristas_Ego.items():
    #print(clave[0],',',valor)
    Ego.add_edge(clave[0],clave[1], value = (valor),label=str(valor)) 

selected_nodes = [n for n,v in G.nodes(data=True) if v['value'] >= tamaño_nodo and n!='mathematics' and n!='education' and n!='nan' and n!='mathematical']
Ego = Ego.subgraph(selected_nodes)


hub_ego = nx.ego_graph(Ego, ego)

net=Network(notebook=True)
net.repulsion()
net.from_nx(hub_ego)
nombre_archivo = 'grafo_ego_'+ego+'_'+ abrev + '_' + campo_para_nodos_grafo + '_met_'+ met +str(len(dfA)) +'_n_' + str(len(df1.index)) + '_f_nodo_'+ str(tamaño_nodo) + '_f_edge_'+ str(filtro_arista)
net.show(abrev + '/'+nombre_archivo+'.html')
lista_aristas_hub_ego=[e for e in hub_ego.edges(data=True)]
lista_aristas_hub_ego = sorted(lista_aristas_hub_ego, key=lambda x: x[2]['value'], reverse=True)


print('\caption{','Ego del nodo',ego, ' para ',abrev, 'que incluye' ,len(dfA), 
      ' artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos ',str(tamaño_nodo),
      ' y las aristas de al menos tamaño ',filtro_arista,'.}')


print('Grafo de la revista: ',abrev)
print('Cantidad de artículos: ',len(df1))
print('Cantidad de nodos grafo completo G: ',(len(hub_ego.nodes)))
print('Cantidad de aristas grafo completo G: ',(len(hub_ego.edges)))


print('\n')
print('Lista de aristas y pesos del EGO: ',ego)
for x in lista_aristas_hub_ego:
    print(x[0],'-',x[1],':',x[2].get('value'))





Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
\caption{ Ego del nodo teacher  para  RELIME-RIITMA que incluye 220  artículos. En este grafo se muestran solo los nodos de cuyo tamaño es al menos  5  y las aristas de al menos tamaño  8 .}
Grafo de la revista:  RELIME-RIITMA
Cantidad de artículos:  220
Cantidad de nodos grafo completo G:  4
Cantidad de aristas grafo completo G:  3


Lista de aristas y pesos del EGO:  teacher
teacher - training : 12
teacher - knowledge : 10
teacher - teaching : 10


### 9.7.1 Exportar a Datapane

nodos G: 1742
aristas G: 3264

nodos H con n= 1  y  a= 1 :  159
aristas H con n= 1  y  a= 1 :  2317

nodos H con n= 2  y  a= 2 :  47
aristas H con n= 2  y  a= 2 :  329

nodos H con n= 3  y  a= 3 :  23
aristas H con n= 3  y  a= 3 :  90

# 10 Resúmenes del grafo

import aspose.words as aw

doc = aw.Document()
builder = aw.DocumentBuilder(doc)

shape = builder.insert_image("Input.png")
shape.image_data.save("Output.svg")

In [44]:
fin_todo_el_proceso = time.time()
print(fin_todo_el_proceso-inicio_todo_el_proceso, ' seg. Equivalentes a ',(fin_todo_el_proceso-inicio_todo_el_proceso)/60, ' min') 


17.41779613494873  seg. Equivalentes a  0.2902966022491455  min


In [45]:
entorno_section_tex(ruta_archivo, nombre_archivo, 'Tiempo total')
texto_tiempo_total='Tiempo requerido para generar todo el proceso del grafo fue de: '+ str(round((fin_todo_el_proceso-inicio_todo_el_proceso)/60,1))+ ' min.'
parrafo_tex(ruta_archivo, nombre_archivo_tex, texto_tiempo_total)


print(texto_tiempo_total)


          

Tiempo requerido para generar todo el proceso del grafo fue de: 0.3 min.


In [46]:
print('Información del grafo:')
print('Grafo de la revista: ',abrev)
print('Cantidad de artículos: ',len(df1))
print('Cantidad de nodos grafo completo G: ',(len(G.nodes)))
print('Cantidad de aristas grafo completo G: ',(len(G.edges)))
print(dic_nombres_campo.get(dic_campo_a_analizar.get(campo_a_analizar)))


Información del grafo:
Grafo de la revista:  RELIME-RIITMA
Cantidad de artículos:  220
Cantidad de nodos grafo completo G:  578
Cantidad de aristas grafo completo G:  4713
PALABRAS CLAVE


In [47]:
#print((G.nodes(data=True)))
print('\n')
#print((G.edges(data=True)))
#print(G['technology'])
#print((G.nodes))
lista_nodos= [e for e in G.nodes(data=True)]


lista_aristas=[e for e in G.edges(data=True)]
print('\n')
print(lista_aristas[:10])

#for x in lista_nodos:
#    print(x[1].get('label'))


#print([e for e in G.nodes(data=True)][2])
#print('\n')
#print([e for e in G.nodes(data=True)][2][0])
print('\n')
print('\n')
#print([(u,v,e) for (u,v,e) in G.edges(data=True) if e['value']==1])
lista_nodos[2][1]


parrafo_tex(ruta_archivo, nombre_archivo_tex, "\\newpage \n'")
parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\bibliographystyle{apalike-es.bst}')
parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\bibliography{referencias.bib}')
parrafo_tex(ruta_archivo, nombre_archivo_tex, '\\end{document}')


fin_todo = time.time()
print('')
#print(fin_todo-inicio_todo, ' seg. Equivalentes a ',(fin_todo-inicio_todo)/60, ' min') 






[('mathematics', 'education', {'value': 29, 'label': '29'}), ('mathematics', 'teacher', {'value': 17, 'label': '17'}), ('mathematics', 'teaching', {'value': 14, 'label': '14'}), ('mathematics', 'learning', {'value': 11, 'label': '11'}), ('mathematics', 'knowledge', {'value': 7, 'label': '7'}), ('mathematics', 'history', {'value': 6, 'label': '6'}), ('mathematics', 'mathematical', {'value': 6, 'label': '6'}), ('mathematics', 'technology', {'value': 5, 'label': '5'}), ('mathematics', 'geometry', {'value': 5, 'label': '5'}), ('mathematics', 'preservice', {'value': 5, 'label': '5'})]





